In [1]:
!pip install transformers
!pip install datasets
!pip install pandas

import os
import sys
from google.colab import drive

device='cuda'
drive.mount('/content/drive')
drive_path = 'wav2vec2_final'
final = os.path.join('drive', 'My Drive', drive_path)
print(os.listdir(final))
sys.path.append(final)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 15.9 MB/s 
     |████████████████████████████████| 163 kB 68.8 MB/s 
     |████████████████████████████████| 7.6 MB 46.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 15.5 MB/s 
     |████████████████████████████████| 95 kB 5.9 MB/s 
     |████████████████████████████████| 212 kB 71.6 MB/s 
     |████████████████████████████████| 115 kB 67.0 MB/s 
     |████████████████████████████████| 127 kB 53.5 MB/s 
     |████████████████████████████████| 115 kB 68.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully un

In [2]:
import numpy
import librosa
import torch
from datasets import Dataset
import pandas as pd

file_path = final + '/Data'
paths = []
arr = []

for i in range(29):
  paths.append(file_path + f'/84-121123-00{i}.wav')

for i in range(25):
  paths.append(file_path + f'/1272-135031-00{i}.wav')

file = file_path + '/transcription.txt'

"""for i in range(60):
  paths.append(file_path + f'/{i+1}.wav')

file = file_path + '/text.txt'"""

with open(file) as f:
    transcription = f.readlines()

for i in range(54):
  transcription[i] = transcription[i].strip('\n')
  temp,_ = librosa.load(paths[i])
  arr.append(temp)

In [3]:
dataset = {'array': arr, 
           'sampling_rate': 16000, 
           'path': paths, 
           'transcription': transcription
           }

df = pd.DataFrame(dataset)
dataset = Dataset.from_pandas(df)
print(dataset)

Dataset({
    features: ['array', 'sampling_rate', 'path', 'transcription'],
    num_rows: 54
})


In [4]:
from sklearn.model_selection import train_test_split

dataset = dataset.train_test_split(test_size = 0.2)

In [5]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Tokenizer

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [6]:
def prepare_dataset(batch):
    batch["input_values"] = processor(batch["array"], sampling_rate=batch["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

dataset = dataset.map(prepare_dataset, num_proc=4)

#0:   0%|          | 0/11 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your "


#1:   0%|          | 0/11 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your "


#2:   0%|          | 0/11 [00:00<?, ?ex/s]

...

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your "


#2:   0%|          | 0/3 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your "


#3:   0%|          | 0/2 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  "`as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your "


In [7]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch
        
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [8]:
from transformers import Wav2Vec2ForCTC, TrainingArguments, Trainer

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base-960h",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

Downloading:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    group_by_length=True,
    per_gpu_train_batch_size=8,
    evaluation_strategy="steps",
    num_train_epochs=3,
    fp16=True,
    gradient_checkpointing=True,
    learning_rate=1e-4,
    weight_decay=0.005,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
)

Using cuda_amp half precision backend
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [10]:
torch.cuda.empty_cache()
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length, path, sampling_rate, array, transcription. If input_length, path, sampling_rate, array, transcription are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW i

Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=18, training_loss=1.2248768276638455, metrics={'train_runtime': 34.6409, 'train_samples_per_second': 3.724, 'train_steps_per_second': 0.52, 'total_flos': 1.911081836175264e+16, 'train_loss': 1.2248768276638455, 'epoch': 3.0})

In [11]:
audio_input = dataset['test']['array'][0]
input_values = processor(audio_input, sampling_rate = 16000, return_tensors="pt",device='cuda',padding=True).input_values

logits = model(input_values.cuda()).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0])
transcription

'I IAM GOING SIR AND I DO NOT HHESITAYEE TO SAY THAT NO PRAYYERS WILL BE MORE FERVENTTA MINE'

In [12]:
dataset['test']['transcription'][0]

'I AM GOING SIR AND I DO NOT HESITATE TO SAY THAT NO PRAYERS WILL BE MORE FERVENT THAN MINE'